In [7]:
import os
import numpy as np
import pandas as pd
import glob
import sys
sys.path.append("../common_tools/peaks_confusion_matrix")
from confusion_matrix import compute_confusion_matrix
from plot_false_peaks import plot_fp_fn

In [8]:
def terma_detect_peaks(input_dir, output_dir, w_cycle=55, w_evt=9, beta=0.095):
    """
    Processes multiple PPG files, detects peaks, and saves the results into CSV files.
    
    Parameters:
    - input_dir: Path to the directory containing input PPG files.
    - output_dir: Path to the directory where the output files will be saved.
    - w_cycle: Window size for calculating the cycle moving average.
    - w_evt: Window size for calculating the event moving average.
    - beta: Scaling factor for the threshold.
    """
    
    # Loop over all files in the input directory
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.csv'):  # Process only CSV files
            input_file = os.path.join(input_dir, file_name)
            print(f"Processing file: {file_name}")
            
            # Read the PPG data from the file
            ppg_data = pd.read_csv(input_file, header=None)
            signal = np.asarray(ppg_data.iloc[:, 0], dtype=np.float64)  # Assume the PPG data is in the first column
            
            # Shift the signal to eliminate negative values
            min_signal = np.min(signal)
            if min_signal < 0:
                signal = signal - min_signal  # Shift signal so the minimum value becomes 0 or greater

            # Enhance the signal (square the values)
            signal = signal * signal
            
            # Initialize arrays for moving averages and thresholds
            ma_cycle = np.zeros_like(signal, dtype=np.float64)
            ma_evt = np.zeros_like(signal, dtype=np.float64)
            mean_signal = np.mean(signal)
            mean = np.full(len(signal), mean_signal)
            threshold_1 = np.zeros_like(signal, dtype=np.float64)
            block_of_interest = np.zeros_like(signal, dtype=np.float64)
            peak_index_arr = []
            
            # Calculate the Event Duration Moving Average
            for i in range((w_evt - 1) // 2, len(signal) - ((w_evt - 1) // 2)):
                ma_evt[i] = np.mean(signal[i - (w_evt - 1) // 2 : i + (w_evt - 1) // 2 + 1])
            
            # Calculate the Event Cycle Moving Average
            for i in range((w_cycle - 1) // 2, len(signal) - ((w_cycle - 1) // 2)):
                ma_cycle[i] = np.mean(signal[i - (w_cycle - 1) // 2 : i + (w_cycle - 1) // 2 + 1])
            
            # Calculate the Threshold for Block of Interest
            for i in range(len(signal)):
                threshold_1[i] = ma_cycle[i] + beta * mean[i]
            
            # Generate the Block of Interest
            for i in range(len(signal)):
                if (ma_evt[i] > threshold_1[i]) and (ma_cycle[i] != 0):
                    block_of_interest[i] = 1
                else:
                    block_of_interest[i] = 0
            
            # Peak detector logic
            start_block = 0
            stop_block = 0
            peak = 0
            peak_index = 0
            block_num = 0
            
            for i in range(len(block_of_interest) - 1):
                if (block_of_interest[i + 1] - block_of_interest[i]) == 1:
                    start_block = i
                if (block_of_interest[i] - block_of_interest[i + 1]) == 1:
                    stop_block = i
                    if (stop_block - start_block) >= w_evt:
                        peak = signal[start_block]
                        for j in range(start_block, stop_block + 1):
                            if signal[j] > peak:
                                peak = signal[j]
                                peak_index = j
                        peak_index_arr.append(peak_index)
                        block_num += 1
            
            # Save the peak indices
            output_file = os.path.join(output_dir, f"{file_name.replace('.csv', '')}_detected_peaks.csv")
            pd.DataFrame(peak_index_arr).to_csv(output_file, header=False, index=False)
            print(f"Peaks saved to: {output_file}")


In [9]:
test_data_dir = 'E:/dilated_cnn_peak_detection_model_data/test/test_data/data'
label_dir = 'E:/dilated_cnn_peak_detection_model_data/test/test_data/label'
output_dir = 'E:/dilated_cnn_peak_detection_model_data/test/algorithm_detected_peaks/'
plot_output_dir = 'E:/dilated_cnn_peak_detection_model_data/test/algorithm_plot_results/'

# Create directory to save results
os.makedirs(output_dir, exist_ok=True)
os.makedirs(plot_output_dir, exist_ok=True)

for file in glob.glob(os.path.join(output_dir, "*.csv")):
    os.remove(file)

for file in glob.glob(os.path.join(plot_output_dir, "*.png")):
    os.remove(file)

In [10]:
terma_detect_peaks(test_data_dir, output_dir)

Processing file: golden_ppg_data_100hz.csv
Peaks saved to: E:/dilated_cnn_peak_detection_model_data/test/algorithm_detected_peaks/golden_ppg_data_100hz_detected_peaks.csv
Processing file: p000946-2120-05-14-08-08.csv
Peaks saved to: E:/dilated_cnn_peak_detection_model_data/test/algorithm_detected_peaks/p000946-2120-05-14-08-08_detected_peaks.csv
Processing file: p025117-2202-03-17-09-14.csv
Peaks saved to: E:/dilated_cnn_peak_detection_model_data/test/algorithm_detected_peaks/p025117-2202-03-17-09-14_detected_peaks.csv
Processing file: p050384-2195-01-30-02-21.csv
Peaks saved to: E:/dilated_cnn_peak_detection_model_data/test/algorithm_detected_peaks/p050384-2195-01-30-02-21_detected_peaks.csv
Processing file: p063628-2176-07-05-17-12.csv
Peaks saved to: E:/dilated_cnn_peak_detection_model_data/test/algorithm_detected_peaks/p063628-2176-07-05-17-12_detected_peaks.csv
Processing file: p075796-2198-07-25-22-55.csv
Peaks saved to: E:/dilated_cnn_peak_detection_model_data/test/algorithm_det

In [11]:
# Test and save results
for file_name in os.listdir(test_data_dir):
    input_file = os.path.join(test_data_dir, file_name)
    ppg_data = pd.read_csv(input_file, header=None).squeeze("columns").values

    labeled_file = os.path.join(label_dir, file_name.replace(".csv", "_labeled_peaks.csv"))
    detected_file = os.path.join(output_dir, file_name.replace(".csv", "_detected_peaks.csv"))
    
    if os.path.exists(labeled_file):
        labeled_peaks = pd.read_csv(labeled_file, header=None).squeeze("columns").values
        detected_peaks = pd.read_csv(detected_file, header=None).squeeze("columns").values

        # Compute confusion matrix
        results = compute_confusion_matrix(detected_peaks, labeled_peaks)
        print(f"File: {file_name}")
        print(f"True Positives (TP): {results['tp']}")
        print(f"False Positives (FP): {results['fp']}")
        print(f"False Negatives (FN): {results['fn']}")
        print(f"Accuracy: {results['accuracy']:.2f}")
        print(f"Precision: {results['precision']:.2f}")
        print(f"Recall: {results['recall']:.2f}")
        print(f"Detection Error Rate: {results['der']:.2f}")
        print("False Positives (FP) at indices:", results['fp_list'])
        print("False Negatives (FN) at indices:", results['fn_list'])
        print("")

        # Plot False Positives and False Negatives
        # plot_fp_fn(ppg_data, results['fp_list'], results['fn_list'], detected_peaks, file_name, plot_output_dir)

File: golden_ppg_data_100hz.csv
True Positives (TP): 272
False Positives (FP): 1
False Negatives (FN): 0
Accuracy: 1.00
Precision: 1.00
Recall: 1.00
Detection Error Rate: 0.00
False Positives (FP) at indices: [11727]
False Negatives (FN) at indices: []

File: p000946-2120-05-14-08-08.csv
True Positives (TP): 31159
False Positives (FP): 902
False Negatives (FN): 166
Accuracy: 0.97
Precision: 0.97
Recall: 0.99
Detection Error Rate: 0.03
False Positives (FP) at indices: [69, 138, 192, 261, 466, 881, 1029, 8125, 8355, 8706, 9229, 11781, 22466, 36433, 38590, 38679, 38925, 39355, 41569, 56236, 62199, 62323, 79475, 82847, 83144, 83161, 84991, 87720, 87769, 88189, 88281, 88998, 89098, 89261, 89377, 91171, 92542, 92622, 96008, 96008, 96337, 96475, 96510, 102643, 105610, 105732, 105904, 127791, 134379, 138463, 138555, 139398, 139489, 140546, 140825, 145436, 145915, 145998, 146451, 146481, 146634, 146718, 146811, 149916, 157346, 157608, 157655, 157705, 158557, 158676, 158884, 159198, 159262, 1593